In [22]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [2]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [3]:
from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model="llama3:latest")
# llm = ChatOllama(model="Llama-3-Open-Ko-8B-FP16.gguf:latest")
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('../assets/news.txt')
data = loader.load()

print(f'type : {type(data)} / len : {len(data)}')
print(f'data : {data}')
print(f'page_content : {data[0].page_content}')

type : <class 'list'> / len : 1
data : [Document(page_content="엔비디아가 인공지능(AI) 모델을 개발하거나 실행하는 비용을 낮추기 위해 관련 스타트업을 잇달아 인수했다는 소식이다. GPU 생태계 강화 수단을 추가, AI 지배력을 유지하는 데 도움이 될 수 있다는 분석이다.\n\n디 인포메이션은 24일(현지시간) 엔비디아가 이스라엘 AI 스타트업 런콜론ai(Run:ai) 및 데시(Deci)와 두건의 인수 계약을 체결했다고 보도했다.\n\n이에 따르면 150여명의 직원을 보유한 런콜론ai는 엔비디아로부터 7억달러(약 9700억원)에 인수됐다. 이에 앞서 이 회사는 인사이트 파트너스 및 타이거 글로벌 매니지먼트를 포함한 투자자로부터 1억1800만달러(약 1600억원)의 자금을 조달한 바 있다.\n\n반면 데시의 인수 가격은 알려지지 않았다. 데시는 인사이트 파트너스 및 스퀘어 패그 캐피털과 같은 투자자로부터 5500만달러(약 760억원)를 모금했으며, 약 100명의 직원을 보유하고 있다.\n\n두 스타트업은 모두 생성 AI 모델 개발이나 실행 비용을 낮추는 것을 목표로 하고 있다. AI 모델을 구동하는 GPU 칩에 대한 수요를 유지하려는 엔비디아의 노력과도 부합한다.\n\n우선 런콜론ai는 개발자가 여러 AI 워크로드를 순차적이 아닌 병렬로 실행할 수 있도록 만들어, AI 칩을 효율적으로 만든다. 이는 결과적으로 필요한 GPU의 수를 줄이는 데 도움이 된다.\n\nGPU나 다른 AI 가속기에서 실행되는 AI 워크로드에 맞춰 필요한 컴퓨팅 용량을 자동으로 할당, GPU 자원을 효과적으로 공유할 수 있도록 지원하는 쿠버네티스 기반 컨테이너 플랫폼을 개발한다.\n\n특히 GPU 클러스터 리소스를 여러 AI 워크로드에 공유하기 위해 AI '가상화 인프라'를 구축한다는 것이 특징이다. 이를 활용하면 컴퓨팅에 가상 계층을 구축, 적은 리소스를 쓰면서 빠르게 GPU에서 AI 모델을 학습할 수 있다.\n\n데시는 AI 칩에서 더 

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf")
pages = loader.load()

print(f'type : {type(pages)} / len : {len(pages)} / pages : {pages}')

type : <class 'list'> / len : 227 / pages : [Document(page_content=' \n연구책임자 : \n김\n 주\n 희(국 립 부 경 대 학 교)\n공동연구자 :\n차\n 재\n 권(국 립 부 경 대 학 교)\n김\n 현\n 정(동 아 대 학 교)\n조\n 성\n 복(국 민 대 학 교)\n연구보조원 : \n박\n 서\n 현(국 립 부 경 대 학 교)\n권\n 수\n 민(국 립 부 경 대 학 교)\n국립부경대학교 산학협력단\n2023년도 중앙선거관리위원회 정책연구용역 보고서\n생성형 AI 신기술 도입에 따른 \n선거 규제 연구\n', metadata={'source': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_path': '../assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'page': 0, 'total_pages': 227, 'format': 'PDF 1.4', 'title': '', 'author': 'fpqlt', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.9139', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20231228104945+09'00'", 'modDate': "D:20231228104945+09'00'", 'trapped': ''}), Document(page_content='ii\n【2023년도 중앙선거관리위원회 정책연구용역 보고서】\n『생성형 AI 신기술 도입에 따른 \n선거 규제 연구』\n연구책임자 : \n김\n 주\n 희(국 립 부 경 대 학 교)\n공동연구자 :\n차\n 재\n 권(국 립 부 경 대 학 교)\n김\n 현\n 정(동 아 대 학 교)\n조\n 성\n 복(국 민 대 학 교)\n연구보조원 : \n박\n 서\n 현(국 립 부 경 대 학 교)\n권\n 수\n 민(국 립 부 경

In [9]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# BeautifulSoup : HTML 및 XML 문서를 파싱하고 구문 분석하는 데 사용되는 파이썬 라이브러리. 주로 웹 스크레이핑(웹 페이지에서 데이터 추출) 작업에서 사용되며, 웹 페이지의 구조를 이해하고 필요한 정보를 추출하는 데 유용
loader = WebBaseLoader(
    web_paths=("https://www.aitimes.com/news/articleView.html?idxno=159102"
               , "https://www.aitimes.com/news/articleView.html?idxno=159072"
               , "https://www.aitimes.com/news/articleView.html?idxno=158943"
               ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "article", # 태그
            attrs={"id": ["article-view-content-div"]}, # 태그의 ID 값들
        )
    ),
)
data = loader.load()

print(f'type : {type(data)} / len : {len(data)}')
print(f'data : {data}')
for d in data:
    print(f'page_content : {d.page_content}')

type : <class 'list'> / len : 3
data : [Document(page_content="\n\n\n(사진=엔비디아)\n\n\n엔비디아가 인공지능(AI) 모델을 개발하거나 실행하는 비용을 낮추기 위해 관련 스타트업을 잇달아 인수했다는 소식이다. GPU 생태계 강화 수단을 추가, AI 지배력을 유지하는 데 도움이 될 수 있다는 분석이다.\n디 인포메이션은 24일(현지시간) 엔비디아가 이스라엘 AI 스타트업 런콜론ai(Run:ai) 및 데시(Deci)와 두건의 인수 계약을 체결했다고 보도했다.\n이에 따르면\xa0150여명의 직원을 보유한 런콜론ai는 엔비디아로부터 7억달러(약 9700억원)에 인수됐다. 이에 앞서 이 회사는 인사이트 파트너스 및 타이거 글로벌 매니지먼트를 포함한 투자자로부터 1억1800만달러(약 1600억원)의 자금을 조달한 바 있다.\n반면 데시의 인수 가격은 알려지지 않았다. 데시는 인사이트 파트너스 및 스퀘어 패그 캐피털과 같은 투자자로부터 5500만달러(약 760억원)를 모금했으며, 약 100명의 직원을 보유하고 있다.\n두 스타트업은 모두 생성\xa0AI 모델\xa0개발이나 실행\xa0비용을 낮추는 것을 목표로 하고 있다. AI 모델을 구동하는 GPU 칩에 대한 수요를 유지하려는 엔비디아의 노력과도 부합한다.\n우선 런콜론ai는 개발자가 여러 AI 워크로드를 순차적이 아닌 병렬로 실행할 수 있도록 만들어, AI 칩을 효율적으로 만든다. 이는 결과적으로 필요한 GPU의 수를 줄이는 데 도움이 된다.\nGPU나 다른 AI 가속기에서 실행되는 AI 워크로드에 맞춰 필요한 컴퓨팅 용량을 자동으로 할당, GPU 자원을 효과적으로 공유할 수 있도록 지원하는 쿠버네티스 기반 컨테이너 플랫폼을 개발한다.\n특히 GPU 클러스터 리소스를 여러 AI 워크로드에 공유하기 위해 AI '가상화 인프라'를 구축한다는 것이 특징이다. 이를 활용하면 컴퓨팅에 가상 계층을 구축, 적은 리소스를 쓰면서 빠르게 GPU에서 AI 모델을 학습할 수 있

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(data)
print(f'len(splits[0].page_content) : {len(splits[1].page_content)}')
splits

len(splits[0].page_content) : 456


[Document(page_content='(사진=엔비디아)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=159102'}),
 Document(page_content='엔비디아가 인공지능(AI) 모델을 개발하거나 실행하는 비용을 낮추기 위해 관련 스타트업을 잇달아 인수했다는 소식이다. GPU 생태계 강화 수단을 추가, AI 지배력을 유지하는 데 도움이 될 수 있다는 분석이다.\n디 인포메이션은 24일(현지시간) 엔비디아가 이스라엘 AI 스타트업 런콜론ai(Run:ai) 및 데시(Deci)와 두건의 인수 계약을 체결했다고 보도했다.\n이에 따르면\xa0150여명의 직원을 보유한 런콜론ai는 엔비디아로부터 7억달러(약 9700억원)에 인수됐다. 이에 앞서 이 회사는 인사이트 파트너스 및 타이거 글로벌 매니지먼트를 포함한 투자자로부터 1억1800만달러(약 1600억원)의 자금을 조달한 바 있다.\n반면 데시의 인수 가격은 알려지지 않았다. 데시는 인사이트 파트너스 및 스퀘어 패그 캐피털과 같은 투자자로부터 5500만달러(약 760억원)를 모금했으며, 약 100명의 직원을 보유하고 있다.', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=159102'}),
 Document(page_content="두 스타트업은 모두 생성\xa0AI 모델\xa0개발이나 실행\xa0비용을 낮추는 것을 목표로 하고 있다. AI 모델을 구동하는 GPU 칩에 대한 수요를 유지하려는 엔비디아의 노력과도 부합한다.\n우선 런콜론ai는 개발자가 여러 AI 워크로드를 순차적이 아닌 병렬로 실행할 수 있도록 만들어, AI 칩을 효율적으로 만든다. 이는 결과적으로 필요한 GPU의 수를 줄이는 데 도움이 된다.\nGPU나 다른 AI 가속기에서 실행되는 AI 워크로드에 맞춰 필요한 컴퓨팅 용량을 자동으로 할당, GPU 자원을 효과적으

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)
embed = embeddings.embed_documents(
    [
        "안녕 영광",
        "동해물과 백두산",
        "마르고 닳도록",
        "하느님이 보우하사",
        "우리나라 만세"
    ]
)
print(f"len(embed): {len(embed)}")
print(f"len(embed[0]): {len(embed[0])}")
print(f"len(embed[1]): {len(embed[1])}")
print(f"embed: {embed}")

len(embed): 5
len(embed[0]): 1024
len(embed[1]): 1024
embed: [[-0.04406510666012764, 0.02941668964922428, -0.0075853848829865456, -0.0205543152987957, -0.05338962748646736, -0.040562961250543594, -0.03600797429680824, -0.025920692831277847, 0.041742049157619476, -0.006995216943323612, 0.04310370236635208, 0.031573522835969925, -0.0061150058172643185, 0.014284436590969563, 0.04593217745423317, -0.016141299158334732, 0.04716397449374199, -0.012035963125526905, -0.02962535433471203, 0.030895672738552094, -0.03735633194446564, -0.016930188983678818, 0.013374143280088902, 0.0003796721575781703, 0.014378626830875874, 0.0286861639469862, -0.001423727604560554, -0.0069765676744282246, 0.02481277473270893, -0.03468279913067818, 0.01399201713502407, 0.0381195992231369, -0.008146059699356556, -0.009772093035280704, -0.0641954243183136, -0.046275701373815536, 0.01252890843898058, 0.004261685535311699, -0.02226465940475464, 0.06601041555404663, 0.06107230857014656, -0.01947237364947796, 0.007392176

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

vectorstore = FAISS.from_documents(splits,
                                   embedding = embeddings,
                                   distance_strategy = DistanceStrategy.COSINE # 코사인 유사도 측정. 값이 클수록 더 유사함을 의미
                                  )

# 로컬에 DB 저장
MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore.save_local(MY_FAISS_INDEX)

vectorstore

In [25]:
# 로컬 DB 불러오기
MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore = FAISS.load_local(MY_FAISS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출
retrieved_docs = retriever.invoke("라마3")
retrieved_docs

[Document(page_content='특히 15조개 이상의 토큰을 동원, 학습량이 라마 2 대비 7배 이상 많으며 코드량은 4배 더 많다. 다만 데이터셋은 공개하지 않았다.\n이후 미세조정에는 일상적인 질문부터 과학·기술·공학·수학(STEM), 코딩, 역사 지식에 이르기까지 다양한 분야의 데이터셋이 사용됐다. 훈련\xa0규모를 확대하는 것은 물론, 고도화된 ‘지시 미세조정(instruction fine-tuning)’ 과정도 진행했다.\xa0\n또\xa0라마 3는 라마 2보다 2배 큰 8000토큰의 컨텍스트 길이를 지원한다.\n오픈 소스라는 점을 감안, 안전하고 책임감 있는 개발과 사용을 위한 다양한 안전장치도 마련했다고 밝혔다. 전문가와 자동화된 도구를 활용한 레드팀 테스트를 통해 부적절한 답변의 가능성을 최소화했다고 전했다.', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='라마 3 인간 평가 결과 (사진=메타)\n\n\n허깅페이스에 따르면, 라마 3는 공개 후 몇시간만에 LLM 리더보드\xa01위에 오르며 역대 가장 빠른 1위 달성 기록을 세웠다.\n또 이전 라마 1과 2를 기반으로 3만개 이상의 새로운 모델이 출시됐으며, 라마 2 모델은 1700억번 다운로드됐다는 통계치도 공개해 눈길을 모았다.\xa0\n다만 라마 3는 완전한 오픈 소스가 아니다.\xa0연구용 및 상업용으로 모두 사용할 수 있지만, 개발자가 다른 생성 모델을 훈련하기 위해 모델을 사용하는 것을 금지한다.\n\n\n메타 AI (사진=메타)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content="라마 3 벤치마크 결과 (사진=메타)\n\n\n라마 3는 객관식 문제(MMLU)와 코딩(HumanEval)에

In [24]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

# prompt = hub.pull("rlm/rag-prompt") # https://smith.langchain.com/hub/rlm/rag-prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", """
너는 유능한 업무 보조자야.
다음 context를 사용해서 question에 대한 답을 말해줘. 답에 대한 출처가 있다면 반드시 출처를 명시해줘.
정답을 모르면 모른다고만 해.

# question : {question}

# context ========================
{context}
==================================

# answer :
"""
    ),
])

retrieved_docs = []
def get_relevant_documents(query):
    global retrieved_docs
    retrieved_docs = retriever.invoke(query)
    return retrieved_docs

def get_page_contents_with_metadata(docs) -> str: 
    result = ""
    for i, doc in enumerate(docs):
        if i > 0:
            result += "\n\n"
        result += f"## 본문: {doc.page_content}\n### 출처: {doc.metadata['source']}"
    return result

def find_docs_and_get_contents(query) -> str:
    docs = get_relevant_documents(query)
    return get_page_contents_with_metadata(docs)

def get_metadata_sources(docs) -> str: 
    sources = set()
    for doc in docs:
        sources.add(doc.metadata['source'])
    return "\n".join(sources)

def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return f"{ai_message.content}\n\n[출처]\n{get_metadata_sources(retrieved_docs)}"

chain = (
    {"context": find_docs_and_get_contents, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parse
)

result = chain.invoke("퍼플렉시티가 투자받은 금액?")
print(f"{result}")

퍼플렉시티는 최근 약 6300만 달러(약 86억 원)의 투자를 유치했으며, 이로 인해 회사 가치는 10억 달러(약 1조 3760억 원) 이상으로 평가되었습니다.

[출처]
https://www.aitimes.com/news/articleView.html?idxno=159072
https://www.aitimes.com/news/articleView.html?idxno=159102
